In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

## Predicting that a coffee is good or not?

In [2]:
# Training Set
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

x_train, y_train = load_coffee_data()

print(f"x_shape={x_train.shape}, y_shape={y_train.shape}")


x_shape=(200, 2), y_shape=(200, 1)


## Before implementing the model, first we have to normalise it

In [3]:
# x_train[:, 0] # this will give me the array with first column data only
print(f"Temperature Max, Min pre normalization: {np.max(x_train[:,0]):0.2f}, {np.min(x_train[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(x_train[:,1]):0.2f}, {np.min(x_train[:,1]):0.2f}")
normal_l = tf.keras.layers.Normalization()
normal_l.adapt(x_train) # learns mean and varience
xn_train = normal_l(x_train)
print(f"Temperature Max, Min post normalization: {np.max(xn_train[:,0]):0.2f}, {np.min(xn_train[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(xn_train[:,1]):0.2f}, {np.min(xn_train[:,1]):0.2f}")

Temperature Max, Min pre normalization: 284.99, 151.32
Duration    Max, Min pre normalization: 15.45, 11.51
Temperature Max, Min post normalization: 1.66, -1.69
Duration    Max, Min post normalization: 1.79, -1.70


Tile/copy our data to increase the training set size and reduce the number of training epochs.

In [4]:
xt_train = np.tile(xn_train, (1000, 1))  # this will create duplecates data 1000 times more 
yt_train = np.tile(y_train, (1000, 1))

print(f"xt_train.shape = {xt_train.shape}, yt_train.shape = {yt_train.shape}")

xt_train.shape = (200000, 2), yt_train.shape = (200000, 1)


## Tensorflow Model

In [5]:
model = Sequential([
    tf.keras.Input(shape=(2,)),
    Dense(units=3, activation="sigmoid", name="layer1"),  # hidden layer
    Dense(units=1, activation="sigmoid", name="layer2")   # output layer
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13 (52.00 Byte)
Trainable params: 13 (52.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
L1_num_params = 2 * 3 + 3  
L2_num_params = 3 * 1 + 1
print("L1 params = ", L1_num_params, ", L2 params = ", L2_num_params  )

L1 params =  9 , L2 params =  4



<pre>
Let's examine the weights and biases Tensorflow has instantiated. The weights
should be of size (number of features in input, number of units in the layer) while the bias
size should match the number of units in the layer:

     -> In the first layer with 3 units, we expect W to have a size of (2,3) and should have 3 elements.
     -> In the second layer with 1 unit, we expect W to have a size of (3,1) andshould have 1 element.
</pre>

In [8]:
w1, b1 = model.get_layer("layer1").get_weights()
w2, b2 = model.get_layer("layer2").get_weights()
print(f"W1{w1.shape}:\n", w1, f"\nb1{b1.shape}:", b1)
print()
print(f"W2{w2.shape}:\n", w2, f"\nb2{b2.shape}:", b2)

W1(2, 3):
 [[-0.731013    0.77222764  1.0929687 ]
 [-1.0322514   0.5378201  -0.5608119 ]] 
b1(3,): [0. 0. 0.]

W2(3, 1):
 [[-0.9041215 ]
 [-0.12271202]
 [ 0.65165055]] 
b2(1,): [0.]


Above are randomly generated weights and bias

In [9]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [10]:
model.fit(xt_train, yt_train, epochs=10)

Epoch 1/10
6250/6250 [==============================] - 17s 2ms/step - loss: 0.1868 - accuracy: 0.9216
Epoch 2/10
6250/6250 [==============================] - 15s 2ms/step - loss: 0.1348 - accuracy: 0.9459
Epoch 3/10
6250/6250 [==============================] - 15s 2ms/step - loss: 0.1250 - accuracy: 0.9489
Epoch 4/10
6250/6250 [==============================] - 15s 2ms/step - loss: 0.1149 - accuracy: 0.9540
Epoch 5/10
6250/6250 [==============================] - 15s 2ms/step - loss: 0.0549 - accuracy: 0.9828
Epoch 6/10
6250/6250 [==============================] - 15s 2ms/step - loss: 0.0172 - accuracy: 0.9987
Epoch 7/10
6250/6250 [==============================] - 15s 2ms/step - loss: 0.0111 - accuracy: 0.9996
Epoch 8/10
6250/6250 [==============================] - 15s 2ms/step - loss: 0.0077 - accuracy: 0.9999
Epoch 9/10
6250/6250 [==============================] - 15s 2ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch 10/10
6250/6250 [==============================] - 15s 2ms/step - l

The first line, Epoch 1/10, describes which epoch the model is currently running. For efficiency, the training data set is broken into 'batches'. The default size of a batch in Tensorflow is 32. There are 200000 examples in our expanded data set or 6250 batches. The notation on the 2nd line 6250/6250 [==== is describing which batch has been executed.

In [11]:
# Updated weights
w1, b1 = model.get_layer("layer1").get_weights()
w2, b2 = model.get_layer("layer2").get_weights()
print(f"W1{w1.shape}:\n", w1, f"\nb1{b1.shape}:", b1)
print()
print(f"W2{w2.shape}:\n", w2, f"\nb2{b2.shape}:", b2)

W1(2, 3):
 [[-10.302093     0.02421251 -17.065557  ]
 [ -0.20933619  -8.321201   -14.3162775 ]] 
b1(3,): [-11.362192 -10.406691  -2.430029]

W2(3, 1):
 [[-42.589046]
 [-38.053837]
 [ 30.425957]] 
b2(1,): [-8.675089]


In [12]:
X_test = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
Xn_test = normal_l(X_test)
prediction = model.predict(Xn_test)

1/1 [==============================] - 0s 159ms/step


In [13]:
print(prediction)

[[9.6740770e-01]
 [1.6548086e-04]]


In [14]:
predict = (prediction >= 0.5).astype(int)
print(predict)

[[1]
 [0]]
